In [1]:
import cv2 as cv
import numpy as np

# Define paths and parameters
input_path = "C:\\Users\\sai\\Desktop\\project\\Advanced Human Pose Recognition and Analysis System\\imahh.jpg"
output_path = "C:\\Users\\sai\\Desktop\\project\\Advanced Human Pose Recognition and Analysis System\\imahh_output.jpg"
threshold = 0.2
inWidth = 368
inHeight = 368

# Define body parts and pose pairs
BODY_PARTS = {"Nose": 0, "Neck": 1, "RShoulder": 2, "RElbow": 3, "RWrist": 4,
              "LShoulder": 5, "LElbow": 6, "LWrist": 7, "RHip": 8, "RKnee": 9,
              "RAnkle": 10, "LHip": 11, "LKnee": 12, "LAnkle": 13, "REye": 14,
              "LEye": 15, "REar": 16, "LEar": 17, "Background": 18}

POSE_PAIRS = [["Neck", "RShoulder"], ["Neck", "LShoulder"], ["RShoulder", "RElbow"],
              ["RElbow", "RWrist"], ["LShoulder", "LElbow"], ["LElbow", "LWrist"],
              ["Neck", "RHip"], ["RHip", "RKnee"], ["RKnee", "RAnkle"], ["Neck", "LHip"],
              ["LHip", "LKnee"], ["LKnee", "LAnkle"], ["Neck", "Nose"], ["Nose", "REye"],
              ["REye", "REar"], ["Nose", "LEye"], ["LEye", "LEar"]]

# Load the pre-trained network
net = cv.dnn.readNetFromTensorflow("graph_opt.pb")

# Read the input image
frame = cv.imread(input_path)
if frame is None:
    print(f"Could not read the input image from {input_path}")
    exit()

frameWidth = frame.shape[1]
frameHeight = frame.shape[0]

# Prepare the frame for the neural network
net.setInput(cv.dnn.blobFromImage(frame, 1.0, (inWidth, inHeight), (127.5, 127.5, 127.5), swapRB=True, crop=False))
out = net.forward()
out = out[:, :19, :, :]  # MobileNet output [1, 57, -1, -1], we only need the first 19 elements

assert (len(BODY_PARTS) == out.shape[1])

points = []
for i in range(len(BODY_PARTS)):
    # Slice heatmap of corresponding body's part.
    heatMap = out[0, i, :, :]

    # Find the global maximum of the heatmap
    _, conf, _, point = cv.minMaxLoc(heatMap)
    x = (frameWidth * point[0]) / out.shape[3]
    y = (frameHeight * point[1]) / out.shape[2]
    # Add a point if its confidence is higher than threshold.
    points.append((int(x), int(y)) if conf > threshold else None)

for pair in POSE_PAIRS:
    partFrom = pair[0]
    partTo = pair[1]
    assert (partFrom in BODY_PARTS)
    assert (partTo in BODY_PARTS)

    idFrom = BODY_PARTS[partFrom]
    idTo = BODY_PARTS[partTo]

    if points[idFrom] and points[idTo]:
        cv.line(frame, points[idFrom], points[idTo], (0, 255, 0), 3)
        cv.ellipse(frame, points[idFrom], (3, 3), 0, 0, 360, (0, 0, 255), cv.FILLED)
        cv.ellipse(frame, points[idTo], (3, 3), 0, 0, 360, (0, 0, 255), cv.FILLED)

# Save the output image
cv.imwrite(output_path, frame)
print(f"Output image saved to {output_path}")

# Display the output image
cv.imshow('OpenPose using OpenCV', frame)
cv.waitKey(0)
cv.destroyAllWindows()


Output image saved to C:\Users\sai\Desktop\project\Advanced Human Pose Recognition and Analysis System\imahh_output.jpg


In [2]:
import cv2 as cv
import numpy as np

# Define paths and parameters
input_path = "C:\\Users\\sai\\Desktop\\project\\Advanced Human Pose Recognition and Analysis System\\vediooo.mp4"
output_path = "C:\\Users\\sai\\Desktop\\project\\Advanced Human Pose Recognition and Analysis System\\vediooo_output1.mp4"
threshold = 0.2
inWidth = 368
inHeight = 368

# Define body parts and pose pairs
BODY_PARTS = {"Nose": 0, "Neck": 1, "RShoulder": 2, "RElbow": 3, "RWrist": 4,
              "LShoulder": 5, "LElbow": 6, "LWrist": 7, "RHip": 8, "RKnee": 9,
              "RAnkle": 10, "LHip": 11, "LKnee": 12, "LAnkle": 13, "REye": 14,
              "LEye": 15, "REar": 16, "LEar": 17, "Background": 18}

POSE_PAIRS = [["Neck", "RShoulder"], ["Neck", "LShoulder"], ["RShoulder", "RElbow"],
              ["RElbow", "RWrist"], ["LShoulder", "LElbow"], ["LElbow", "LWrist"],
              ["Neck", "RHip"], ["RHip", "RKnee"], ["RKnee", "RAnkle"], ["Neck", "LHip"],
              ["LHip", "LKnee"], ["LKnee", "LAnkle"], ["Neck", "Nose"], ["Nose", "REye"],
              ["REye", "REar"], ["Nose", "LEye"], ["LEye", "LEar"]]

# Load the pre-trained network
net = cv.dnn.readNetFromTensorflow("graph_opt.pb")

# Open the video file
cap = cv.VideoCapture(input_path)
if not cap.isOpened():
    print(f"Could not open the input video from {input_path}")
    exit()

# Get video properties
frame_width = int(cap.get(cv.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv.CAP_PROP_FPS))
fourcc = cv.VideoWriter_fourcc(*'mp4v')  # Define the codec for the output video

# Create a VideoWriter object
out = cv.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frameWidth = frame.shape[1]
    frameHeight = frame.shape[0]

    # Prepare the frame for the neural network
    net.setInput(cv.dnn.blobFromImage(frame, 1.0, (inWidth, inHeight), (127.5, 127.5, 127.5), swapRB=True, crop=False))
    out_net = net.forward()
    out_net = out_net[:, :19, :, :]  # MobileNet output [1, 57, -1, -1], we only need the first 19 elements

    assert(len(BODY_PARTS) == out_net.shape[1])

    points = []
    for i in range(len(BODY_PARTS)):
        # Slice heatmap of corresponding body's part
        heatMap = out_net[0, i, :, :]

        # Find the global maximum of the heatmap
        _, conf, _, point = cv.minMaxLoc(heatMap)
        x = (frameWidth * point[0]) / out_net.shape[3]
        y = (frameHeight * point[1]) / out_net.shape[2]
        # Add a point if its confidence is higher than threshold
        points.append((int(x), int(y)) if conf > threshold else None)

    for pair in POSE_PAIRS:
        partFrom = pair[0]
        partTo = pair[1]
        assert(partFrom in BODY_PARTS)
        assert(partTo in BODY_PARTS)

        idFrom = BODY_PARTS[partFrom]
        idTo = BODY_PARTS[partTo]

        if points[idFrom] and points[idTo]:
            cv.line(frame, points[idFrom], points[idTo], (0, 255, 0), 3)
            cv.ellipse(frame, points[idFrom], (3, 3), 0, 0, 360, (0, 0, 255), cv.FILLED)
            cv.ellipse(frame, points[idTo], (3, 3), 0, 0, 360, (0, 0, 255), cv.FILLED)

    # Write the frame to the output video file
    out.write(frame)

    # Optionally, display the frame (you can comment this out if not needed)
    cv.imshow('OpenPose using OpenCV', frame)
    if cv.waitKey(1) & 0xFF == ord('q'):
        break

# Release the VideoCapture and VideoWriter objects
cap.release()
out.release()
cv.destroyAllWindows()

print(f"Output video saved to {output_path}")


Output video saved to C:\Users\sai\Desktop\project\Advanced Human Pose Recognition and Analysis System\vediooo_output1.mp4
